In [1]:
%matplotlib inline


Word Embeddings: Encoding Lexical Semantics
===========================================

Word embeddings are dense vectors of real numbers, one per word in your
vocabulary. In NLP, it is almost always the case that your features are
words! But how should you represent a word in a computer? You could
store its ascii character representation, but that only tells you what
the word *is*, it doesn't say much about what it *means* (you might be
able to derive its part of speech from its affixes, or properties from
its capitalization, but not much). Even more, in what sense could you
combine these representations? We often want dense outputs from our
neural networks, where the inputs are $|V|$ dimensional, where
$V$ is our vocabulary, but often the outputs are only a few
dimensional (if we are only predicting a handful of labels, for
instance). How do we get from a massive dimensional space to a smaller
dimensional space?

How about instead of ascii representations, we use a one-hot encoding?
That is, we represent the word $w$ by

\begin{align}\overbrace{\left[ 0, 0, \dots, 1, \dots, 0, 0 \right]}^\text{|V| elements}\end{align}

where the 1 is in a location unique to $w$. Any other word will
have a 1 in some other location, and a 0 everywhere else.

There is an enormous drawback to this representation, besides just how
huge it is. It basically treats all words as independent entities with
no relation to each other. What we really want is some notion of
*similarity* between words. Why? Let's see an example.

Suppose we are building a language model. Suppose we have seen the
sentences

* The mathematician ran to the store.
* The physicist ran to the store.
* The mathematician solved the open problem.

in our training data. Now suppose we get a new sentence never before
seen in our training data:

* The physicist solved the open problem.

Our language model might do OK on this sentence, but wouldn't it be much
better if we could use the following two facts:

* We have seen  mathematician and physicist in the same role in a sentence. Somehow they
  have a semantic relation.
* We have seen mathematician in the same role  in this new unseen sentence
  as we are now seeing physicist.

and then infer that physicist is actually a good fit in the new unseen
sentence? This is what we mean by a notion of similarity: we mean
*semantic similarity*, not simply having similar orthographic
representations. It is a technique to combat the sparsity of linguistic
data, by connecting the dots between what we have seen and what we
haven't. This example of course relies on a fundamental linguistic
assumption: that words appearing in similar contexts are related to each
other semantically. This is called the `distributional
hypothesis <https://en.wikipedia.org/wiki/Distributional_semantics>`__.


Getting Dense Word Embeddings
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

How can we solve this problem? That is, how could we actually encode
semantic similarity in words? Maybe we think up some semantic
attributes. For example, we see that both mathematicians and physicists
can run, so maybe we give these words a high score for the "is able to
run" semantic attribute. Think of some other attributes, and imagine
what you might score some common words on those attributes.

If each attribute is a dimension, then we might give each word a vector,
like this:

\begin{align}q_\text{mathematician} = \left[ \overbrace{2.3}^\text{can run},
   \overbrace{9.4}^\text{likes coffee}, \overbrace{-5.5}^\text{majored in Physics}, \dots \right]\end{align}

\begin{align}q_\text{physicist} = \left[ \overbrace{2.5}^\text{can run},
   \overbrace{9.1}^\text{likes coffee}, \overbrace{6.4}^\text{majored in Physics}, \dots \right]\end{align}

Then we can get a measure of similarity between these words by doing:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = q_\text{physicist} \cdot q_\text{mathematician}\end{align}

Although it is more common to normalize by the lengths:

\begin{align}\text{Similarity}(\text{physicist}, \text{mathematician}) = \frac{q_\text{physicist} \cdot q_\text{mathematician}}
   {\| q_\text{\physicist} \| \| q_\text{mathematician} \|} = \cos (\phi)\end{align}

Where $\phi$ is the angle between the two vectors. That way,
extremely similar words (words whose embeddings point in the same
direction) will have similarity 1. Extremely dissimilar words should
have similarity -1.


You can think of the sparse one-hot vectors from the beginning of this
section as a special case of these new vectors we have defined, where
each word basically has similarity 0, and we gave each word some unique
semantic attribute. These new vectors are *dense*, which is to say their
entries are (typically) non-zero.

But these new vectors are a big pain: you could think of thousands of
different semantic attributes that might be relevant to determining
similarity, and how on earth would you set the values of the different
attributes? Central to the idea of deep learning is that the neural
network learns representations of the features, rather than requiring
the programmer to design them herself. So why not just let the word
embeddings be parameters in our model, and then be updated during
training? This is exactly what we will do. We will have some *latent
semantic attributes* that the network can, in principle, learn. Note
that the word embeddings will probably not be interpretable. That is,
although with our hand-crafted vectors above we can see that
mathematicians and physicists are similar in that they both like coffee,
if we allow a neural network to learn the embeddings and see that both
mathematicians and physicisits have a large value in the second
dimension, it is not clear what that means. They are similar in some
latent semantic dimension, but this probably has no interpretation to
us.


In summary, **word embeddings are a representation of the *semantics* of
a word, efficiently encoding semantic information that might be relevant
to the task at hand**. You can embed other things too: part of speech
tags, parse trees, anything! The idea of feature embeddings is central
to the field.


Word Embeddings in Pytorch
~~~~~~~~~~~~~~~~~~~~~~~~~~

Before we get to a worked example and an exercise, a few quick notes
about how to use embeddings in Pytorch and in deep learning programming
in general. Similar to how we defined a unique index for each word when
making one-hot vectors, we also need to define an index for each word
when using embeddings. These will be keys into a lookup table. That is,
embeddings are stored as a $|V| \times D$ matrix, where $D$
is the dimensionality of the embeddings, such that the word assigned
index $i$ has its embedding stored in the $i$'th row of the
matrix. In all of my code, the mapping from words to indices is a
dictionary named word\_to\_ix.

The module that allows you to use embeddings is torch.nn.Embedding,
which takes two arguments: the vocabulary size, and the dimensionality
of the embeddings.

To index into this table, you must use torch.LongTensor (since the
indices are integers, not floats).




In [2]:
# Author: Robert Guthrie

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [3]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.LongTensor([word_to_ix["hello"]])
hello_embed = embeds(autograd.Variable(lookup_tensor))
print(hello_embed)

Variable containing:
-2.9718  1.7070 -0.4305 -2.2820  0.5237
[torch.FloatTensor of size 1x5]



An Example: N-Gram Language Modeling
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

Recall that in an n-gram language model, given a sequence of words
$w$, we want to compute

\begin{align}P(w_i | w_{i-1}, w_{i-2}, \dots, w_{i-n+1} )\end{align}

Where $w_i$ is the ith word of the sequence.

In this example, we will compute the loss function on some training
examples and update the parameters with backpropagation.




In [6]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10
# We will use Shakespeare Sonnet 2
test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
# we should tokenize the input, but we will ignore that for now
# build a list of tuples.  Each tuple is ([ word_i-2, word_i-1 ], target word)
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
            for i in range(len(test_sentence) - 2)]
# print the first 3, just so you can see what they look like
print(trigrams[:3])

vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}


class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out)
        return log_probs


losses = []
loss_function = nn.NLLLoss()
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = torch.Tensor([0])
    for context, target in trigrams:

        # Step 1. Prepare the inputs to be passed to the model (i.e, turn the words
        # into integer indices and wrap them in variables)
        context_idxs = [word_to_ix[w] for w in context]
        context_var = autograd.Variable(torch.LongTensor(context_idxs))

        # Step 2. Recall that torch *accumulates* gradients. Before passing in a
        # new instance, you need to zero out the gradients from the old
        # instance
        model.zero_grad()

        # Step 3. Run the forward pass, getting log probabilities over next
        # words
        log_probs = model(context_var)

        # Step 4. Compute your loss function. (Again, Torch wants the target
        # word wrapped in a variable)
        print("Aqui:")
        print(log_probs)
        print(autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))
        loss = loss_function(log_probs, autograd.Variable(
            torch.LongTensor([word_to_ix[target]])))

        # Step 5. Do the backward pass and update the gradient
        loss.backward()
        optimizer.step()

        total_loss += loss.data
    losses.append(total_loss)
print(losses)  # The loss decreased every iteration over the training data!

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]
Aqui:
Variable containing:

Columns 0 to 9 
-4.4976 -4.7090 -4.2959 -4.4168 -4.4247 -4.1748 -4.5000 -4.6714 -4.7830 -4.7873

Columns 10 to 19 
-4.6161 -4.8706 -4.6467 -4.5819 -4.2162 -4.5375 -4.4919 -4.3795 -4.6089 -4.6605

Columns 20 to 29 
-4.6262 -4.9004 -5.1655 -4.5679 -4.5403 -4.5243 -4.9701 -4.9099 -4.6623 -4.2624

Columns 30 to 39 
-4.7256 -4.7841 -5.1036 -4.8016 -5.2189 -4.5850 -4.5578 -4.1879 -4.5940 -4.1050

Columns 40 to 49 
-4.6522 -4.8306 -4.3934 -4.4447 -4.2385 -4.5536 -4.5361 -4.2050 -4.6103 -4.7566

Columns 50 to 59 
-4.6924 -4.3480 -5.0925 -4.8312 -4.3782 -4.0577 -4.7006 -4.9623 -4.6938 -5.0591

Columns 60 to 69 
-4.4941 -4.3185 -4.5957 -4.6284 -4.5323 -4.4679 -4.4046 -4.3823 -4.4535 -4.6223

Columns 70 to 79 
-5.0675 -4.4731 -4.6582 -4.2774 -4.8151 -4.7065 -4.5267 -4.7558 -4.5324 -4.7040

Columns 80 to 89 
-4.4992 -4.5879 -4.6649 -4.6238 -4.4791 -4.9777 -4.5961 -4.3772

Variable containing:

Columns 0 to 9 
-4.3639 -4.4804 -4.4174 -4.5171 -4.5248 -4.2180 -4.7293 -4.3492 -4.3674 -4.4802

Columns 10 to 19 
-4.4841 -4.5122 -4.6753 -4.7061 -4.4478 -4.6494 -4.4199 -4.5564 -4.6416 -4.4917

Columns 20 to 29 
-4.3790 -4.9937 -4.7789 -4.1288 -4.5009 -4.6025 -4.8480 -4.6718 -4.3323 -4.5263

Columns 30 to 39 
-4.9890 -4.6628 -4.7875 -4.5909 -5.0413 -4.5061 -4.8579 -4.3953 -4.3750 -4.6537

Columns 40 to 49 
-4.2200 -4.9587 -4.8526 -4.7705 -4.4823 -4.5517 -4.8113 -4.5104 -4.4910 -4.7241

Columns 50 to 59 
-4.5422 -4.3878 -4.7397 -4.5856 -4.4874 -4.3070 -4.6227 -4.6212 -4.5695 -4.8038

Columns 60 to 69 
-4.4263 -4.3921 -4.5317 -4.6449 -4.5485 -4.6966 -4.2208 -4.6623 -4.3331 -4.5878

Columns 70 to 79 
-4.8325 -4.6911 -4.8323 -4.3867 -4.5483 -4.7328 -4.6506 -5.0079 -4.5853 -4.6809

Columns 80 to 89 
-4.6196 -4.6920 -4.7565 -4.8483 -4.4282 -4.5328 -4.6150 -4.6510 -4.5619 -4.9046

Columns 90 to 96 
-4.6272 -4.4364 -4.6138 -4.7752 -4.6831 -4.5056 -4.4646
[torch.FloatTen

Aqui:
Variable containing:

Columns 0 to 9 
-4.5157 -4.4933 -4.7280 -4.4124 -4.4127 -4.5464 -4.8413 -4.2310 -4.5244 -4.7665

Columns 10 to 19 
-4.4596 -4.3997 -4.5882 -4.8459 -4.3657 -4.5830 -4.2376 -4.6128 -4.6682 -4.6255

Columns 20 to 29 
-4.3572 -4.8971 -4.7073 -4.5299 -4.4255 -4.7375 -4.9516 -4.6807 -4.1494 -4.8244

Columns 30 to 39 
-4.7548 -4.3761 -4.9281 -4.9450 -4.5491 -4.5151 -4.7746 -4.7226 -4.3690 -4.7093

Columns 40 to 49 
-4.7012 -4.6343 -4.7416 -4.8799 -4.1903 -4.6248 -4.5912 -4.4192 -4.5451 -4.5429

Columns 50 to 59 
-4.0450 -4.4600 -4.5679 -4.9163 -4.5768 -4.8206 -4.7341 -4.7196 -4.2253 -4.5676

Columns 60 to 69 
-4.6169 -4.5467 -4.1667 -4.1913 -4.5374 -4.7585 -4.5814 -4.8798 -4.7716 -4.3674

Columns 70 to 79 
-4.6854 -4.9260 -4.2368 -4.4730 -4.8267 -4.7439 -4.4039 -4.9956 -4.6642 -4.5290

Columns 80 to 89 
-4.3856 -4.4894 -5.0148 -4.5659 -4.4445 -4.6318 -5.1548 -4.6305 -4.7851 -4.8932

Columns 90 to 96 
-4.2955 -4.6408 -4.6109 -4.8295 -4.7887 -4.4544 -4.3253
[torch.Fl

Variable containing:

Columns 0 to 9 
-4.1685 -4.7768 -4.6304 -4.5898 -4.3235 -4.6348 -4.6474 -4.6435 -4.4787 -4.8685

Columns 10 to 19 
-4.6702 -4.4585 -4.8773 -4.7093 -4.5104 -4.3439 -4.2605 -4.5823 -4.8299 -4.3990

Columns 20 to 29 
-4.2284 -4.7808 -4.5763 -4.5697 -4.6055 -4.5292 -4.3820 -4.7022 -4.6310 -4.7908

Columns 30 to 39 
-4.7855 -4.5429 -4.6386 -4.8808 -4.5752 -4.4278 -4.4858 -4.5133 -4.6486 -4.6282

Columns 40 to 49 
-4.7888 -4.3331 -4.8767 -4.7089 -4.3925 -4.5881 -4.8729 -4.5436 -4.5793 -4.5712

Columns 50 to 59 
-4.4127 -4.4040 -4.3929 -4.6436 -4.4048 -4.4760 -4.7295 -4.6581 -4.2282 -4.7492

Columns 60 to 69 
-4.6580 -4.3062 -4.2459 -4.7485 -4.7365 -4.6453 -4.1602 -4.8953 -4.7307 -4.6388

Columns 70 to 79 
-4.5381 -4.7333 -4.2789 -4.7773 -4.5579 -4.5115 -4.4902 -4.6085 -4.5345 -4.4728

Columns 80 to 89 
-4.3905 -4.8337 -4.9927 -4.8147 -4.2342 -4.6952 -4.9707 -4.7341 -4.8134 -4.6481

Columns 90 to 96 
-4.4800 -4.6194 -4.6439 -4.6868 -4.6458 -4.6897 -4.5960
[torch.FloatTen

Aqui:
Variable containing:

Columns 0 to 9 
-4.5672 -4.7513 -4.7851 -4.6282 -4.2634 -4.5512 -5.0127 -4.5308 -4.7634 -5.0198

Columns 10 to 19 
-4.9623 -4.5811 -4.5196 -4.8013 -4.6187 -4.6540 -3.9047 -4.6436 -4.8267 -4.6009

Columns 20 to 29 
-4.7795 -5.4245 -5.0428 -4.5517 -4.2209 -4.5358 -5.3671 -4.6168 -4.1691 -4.4454

Columns 30 to 39 
-4.3089 -4.0784 -4.9685 -4.7175 -4.8218 -4.9416 -4.4424 -4.2561 -4.2183 -4.1283

Columns 40 to 49 
-4.9220 -5.0831 -4.5555 -4.7455 -4.1359 -4.4775 -4.8290 -4.5209 -4.8397 -4.4839

Columns 50 to 59 
-4.0860 -4.8644 -4.8137 -5.1278 -4.8803 -4.4117 -4.8355 -4.7257 -4.2736 -4.8780

Columns 60 to 69 
-4.2120 -4.3031 -4.2092 -4.4235 -4.6854 -4.7603 -4.2560 -4.7569 -4.8401 -4.3714

Columns 70 to 79 
-5.1926 -4.4540 -4.5031 -4.7668 -4.7854 -5.0907 -4.1612 -4.9065 -4.5236 -4.3338

Columns 80 to 89 
-4.2131 -4.1009 -5.0348 -4.3591 -4.3853 -4.7055 -4.8381 -4.5319 -4.8327 -4.6659

Columns 90 to 96 
-4.6850 -4.2855 -4.5976 -4.8632 -4.5885 -4.5894 -4.7990
[torch.Fl

...

Aqui:
Variable containing:

Columns 0 to 9 
-4.5467 -4.7470 -4.2782 -4.9195 -4.6320 -4.5181 -4.5536 -4.9521 -4.1492 -4.7622

Columns 10 to 19 
-4.3699 -4.6731 -4.9277 -4.3873 -4.6551 -4.5336 -4.6133 -4.3293 -4.3815 -4.3948

Columns 20 to 29 
-4.4025 -4.8389 -4.3313 -4.9706 -4.6741 -4.5755 -4.4304 -4.7743 -4.7167 -4.6848

Columns 30 to 39 
-5.1303 -4.8159 -5.1194 -4.2828 -4.7848 -4.2267 -4.6599 -4.3325 -4.5003 -4.7466

Columns 40 to 49 
-4.3071 -4.6911 -4.7603 -4.6561 -4.0443 -4.6465 -4.7734 -4.3816 -4.9822 -4.8339

Columns 50 to 59 
-4.6670 -4.3601 -5.0587 -4.8450 -4.3188 -4.8321 -4.8924 -4.4436 -4.1377 -4.9138

Columns 60 to 69 
-4.1846 -4.3380 -4.5941 -4.2406 -5.1347 -4.9915 -4.2541 -4.7770 -4.5736 -4.6534

Columns 70 to 79 
-4.7889 -5.0467 -4.1272 -4.5257 -4.2825 -4.6120 -4.6553 -5.1483 -4.6826 -4.4601

Columns 80 to 89 
-4.2652 -4.3065 -5.2013 -5.3867 -4.4697 -4.8453 -4.9588 -4.8728 -4.7166 -4.4128

Columns 90 to 96 
-4.2874 -4.4427 -4.3625 -4.4509 -4.4853 -4.3712 -4.6414
[torch.Fl

Aqui:
Variable containing:

Columns 0 to 9 
-4.4921 -4.6907 -4.7037 -4.8365 -4.2784 -4.5384 -4.5850 -4.3037 -4.8772 -4.6427

Columns 10 to 19 
-4.5401 -4.6473 -4.6209 -4.8881 -4.2727 -4.7659 -4.2505 -5.0839 -4.7797 -4.5046

Columns 20 to 29 
-4.0812 -4.5761 -4.7990 -4.5748 -4.3087 -4.7028 -4.7460 -4.8693 -4.4284 -4.3072

Columns 30 to 39 
-4.5889 -4.6640 -4.9340 -4.8019 -5.1870 -4.6205 -4.7396 -4.6820 -4.7341 -4.4623

Columns 40 to 49 
-4.7659 -4.4021 -4.3818 -4.4540 -4.4697 -4.5321 -4.5931 -4.2095 -4.5408 -4.5815

Columns 50 to 59 
-4.6675 -4.6279 -4.4306 -4.5565 -4.5539 -4.5664 -4.6362 -4.9172 -4.7273 -4.5354

Columns 60 to 69 
-4.4484 -4.4674 -4.3707 -4.3793 -4.5811 -4.6376 -4.4751 -4.4360 -4.2494 -4.8272

Columns 70 to 79 
-4.6097 -4.8110 -4.5627 -4.5970 -4.5974 -4.6098 -4.2285 -4.6935 -4.4844 -4.5517

Columns 80 to 89 
-4.2213 -4.6659 -4.7337 -4.6554 -4.1793 -4.5226 -4.7543 -4.8015 -4.8624 -4.7737

Columns 90 to 96 
-4.6137 -4.5499 -4.8665 -5.1431 -4.7595 -4.5249 -4.3255
[torch.Fl


Aqui:
Variable containing:

Columns 0 to 9 
-4.3493 -4.9142 -4.8362 -4.9888 -4.5664 -4.6188 -4.6921 -4.1867 -4.5490 -4.5700

Columns 10 to 19 
-4.1820 -4.7203 -4.7380 -4.6799 -4.3202 -4.5117 -4.2407 -4.5677 -4.9529 -4.0923

Columns 20 to 29 
-4.2525 -4.6157 -4.5071 -4.6317 -4.7194 -4.5947 -4.4749 -4.4923 -4.6884 -4.4504

Columns 30 to 39 
-4.3466 -4.5247 -4.8190 -4.8623 -5.0919 -4.8478 -5.0985 -4.7794 -4.5020 -4.8472

Columns 40 to 49 
-4.4967 -4.2900 -4.9665 -5.0917 -4.3822 -4.9687 -4.5720 -4.2929 -4.2872 -4.6892

Columns 50 to 59 
-3.9834 -4.4596 -4.7473 -4.0575 -4.5837 -4.5626 -5.0034 -5.3292 -4.8250 -4.6539

Columns 60 to 69 
-4.7336 -4.1757 -4.5384 -4.5654 -4.6611 -4.4969 -4.5885 -4.7098 -4.6846 -5.1043

Columns 70 to 79 
-5.1190 -4.7980 -4.4261 -4.5999 -4.3885 -4.4387 -4.8385 -4.9962 -4.3379 -4.5172

Columns 80 to 89 
-4.3130 -4.6864 -4.4313 -5.2010 -4.1762 -4.4813 -4.6264 -4.5963 -4.8539 -4.5056

Columns 90 to 96 
-4.1214 -4.9942 -4.5345 -4.8577 -4.5558 -4.3530 -4.6121
[torch.F

Aqui:
Variable containing:

Columns 0 to 9 
-4.5236 -4.8379 -4.5623 -4.5762 -4.5067 -4.2608 -4.4681 -4.6132 -4.0110 -5.3131

Columns 10 to 19 
-4.4259 -4.5665 -4.5856 -4.2722 -4.4303 -4.4725 -4.3018 -4.5993 -4.7502 -4.6409

Columns 20 to 29 
-4.2574 -5.1409 -4.3821 -4.7953 -4.4066 -4.6965 -4.8907 -5.0028 -4.4309 -4.8226

Columns 30 to 39 
-4.6433 -4.5505 -4.7171 -4.6699 -4.4870 -4.5521 -4.6752 -4.7684 -4.5940 -4.2686

Columns 40 to 49 
-5.2999 -4.9445 -4.9011 -4.6268 -4.2546 -4.8385 -4.2723 -4.4698 -4.3988 -5.0294

Columns 50 to 59 
-4.2735 -4.7348 -5.3054 -4.9605 -4.1535 -4.8155 -5.1564 -4.8436 -4.4483 -5.0285

Columns 60 to 69 
-4.5486 -4.1074 -4.4256 -3.9756 -4.6856 -4.2704 -4.2450 -4.5656 -4.6995 -4.8825

Columns 70 to 79 
-5.0462 -4.6119 -4.2867 -4.9328 -4.6673 -4.6784 -4.4769 -4.4917 -4.6369 -4.7206

Columns 80 to 89 
-4.4121 -4.2882 -4.9838 -4.9490 -4.3773 -4.5895 -4.6743 -4.3535 -4.1732 -4.7361

Columns 90 to 96 
-5.0036 -4.8402 -4.4605 -4.5500 -4.5039 -4.5835 -4.8167
[torch.Fl


Aqui:
Variable containing:

Columns 0 to 9 
-3.8927 -4.7272 -4.7876 -4.5328 -4.3958 -4.5752 -4.6819 -4.6249 -4.6603 -4.8270

Columns 10 to 19 
-4.3975 -4.3329 -4.5557 -4.6188 -4.6226 -4.5881 -4.1872 -4.6113 -4.7646 -4.4632

Columns 20 to 29 
-4.5389 -5.0732 -4.7320 -4.4329 -4.4820 -4.5916 -4.7160 -4.7191 -4.4838 -4.4849

Columns 30 to 39 
-4.7280 -4.2090 -4.6664 -4.8004 -4.6467 -4.5484 -4.7180 -4.5076 -4.2192 -4.4706

Columns 40 to 49 
-4.9087 -4.6972 -4.8373 -4.6785 -4.1634 -4.7796 -4.8696 -4.7790 -4.6135 -4.6049

Columns 50 to 59 
-4.5250 -4.7246 -4.7598 -4.5913 -4.4416 -4.6577 -4.7097 -4.6555 -4.4718 -4.8033

Columns 60 to 69 
-4.3484 -4.4932 -4.5005 -4.5737 -4.7327 -4.6966 -4.3766 -4.5800 -4.5825 -4.6676

Columns 70 to 79 
-4.7357 -4.5363 -4.4567 -4.9048 -4.6042 -4.5930 -4.4037 -4.6347 -4.6003 -4.7539

Columns 80 to 89 
-4.4977 -4.2737 -4.9954 -4.4945 -4.3770 -4.4142 -4.6280 -4.8166 -4.5086 -4.6851

Columns 90 to 96 
-4.7379 -4.3720 -4.4550 -4.9775 -4.4074 -4.5522 -4.6787
[torch.F

Exercise: Computing Word Embeddings: Continuous Bag-of-Words
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

The Continuous Bag-of-Words model (CBOW) is frequently used in NLP deep
learning. It is a model that tries to predict words given the context of
a few words before and a few words after the target word. This is
distinct from language modeling, since CBOW is not sequential and does
not have to be probabilistic. Typcially, CBOW is used to quickly train
word embeddings, and these embeddings are used to initialize the
embeddings of some more complicated model. Usually, this is referred to
as *pretraining embeddings*. It almost always helps performance a couple
of percent.

The CBOW model is as follows. Given a target word $w_i$ and an
$N$ context window on each side, $w_{i-1}, \dots, w_{i-N}$
and $w_{i+1}, \dots, w_{i+N}$, referring to all context words
collectively as $C$, CBOW tries to minimize

\begin{align}-\log p(w_i | C) = -\log \text{Softmax}(A(\sum_{w \in C} q_w) + b)\end{align}

where $q_w$ is the embedding for word $w$.

Implement this model in Pytorch by filling in the class below. Some
tips:

* Think about which parameters you need to define.
* Make sure you know what shape each operation expects. Use .view() if you need to
  reshape.




In [ ]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
               raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])


class CBOW(nn.Module):

    def __init__(self):
        pass

    def forward(self, inputs):
        pass

# create your model and train.  here are some functions to help you make
# the data ready for use by your module


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    tensor = torch.LongTensor(idxs)
    return autograd.Variable(tensor)


make_context_vector(data[0][0], word_to_ix)  # example